# 🛩️ Пример работы с моделью Boeing 747

Этот notebook демонстрирует использование линейной продольной модели Boeing 747 в среде TensorAeroSpace.

## 📋 Что мы будем делать:
1. Импортируем необходимые библиотеки
2. Настроим временные параметры и опорный сигнал
3. Создадим и инициализируем среду
4. Выполним один шаг симуляции
5. Проанализируем результаты

## 📚 Импорт библиотек

Загружаем все необходимые модули для работы с моделью самолета:

In [1]:
# Основные библиотеки
import gymnasium as gym
import numpy as np
from tqdm import tqdm

# Модули TensorAeroSpace
from tensoraerospace.envs import LinearLongitudinalB747
from tensoraerospace.utils import generate_time_period, convert_tp_to_sec_tp
from tensoraerospace.signals.standart import unit_step
from tensoraerospace.agent.ihdp.model import IHDPAgent

## ⚙️ Настройка параметров симуляции

Определяем временные параметры и создаем опорный сигнал для управления углом тангажа:

In [2]:
# Параметры дискретизации
dt = 0.01  # Шаг дискретизации (секунды)

# Генерация временного периода
tp = generate_time_period(tn=20, dt=dt)  # 20 секунд симуляции
tps = convert_tp_to_sec_tp(tp, dt=dt)    # Преобразование в секунды
number_time_steps = len(tp)             # Общее количество временных шагов

# Создание ступенчатого опорного сигнала
# Сигнал активируется на 10-й секунде с амплитудой 5 градусов
reference_signals = np.reshape(
    unit_step(degree=5, tp=tp, time_step=10, output_rad=True), 
    [1, -1]
)

print(f'📊 Параметры симуляции:')
print(f'   • Время симуляции: {tp[-1]:.1f} сек')
print(f'   • Шаг дискретизации: {dt} сек')
print(f'   • Количество шагов: {number_time_steps}')

## 🚀 Создание и инициализация среды

Создаем среду Boeing 747 с заданными параметрами и выполняем сброс:

In [3]:
# Создание среды Boeing 747
env = gym.make(
    'LinearLongitudinalB747-v0',
    number_time_steps=number_time_steps,     # Количество временных шагов
    initial_state=[[0],[0],[0],[0]],         # Начальное состояние [u, w, q, theta]
    reference_signal=reference_signals,      # Опорный сигнал
    state_space=['u', 'w', 'q', 'theta'],   # Пространство состояний
    output_space=['u', 'w', 'q', 'theta']   # Пространство выходов
)

# Инициализация среды
state, info = env.reset()

print(f'✅ Среда успешно создана и инициализирована!')
print(f'📈 Начальное состояние: {state.flatten()}')
print(f'🎯 Форма пространства состояний: {env.observation_space.shape}')

/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


## 🎮 Выполнение шага симуляции

Применяем управляющее воздействие и наблюдаем реакцию системы:

In [4]:
# Применение управляющего воздействия
control_input = np.array([[1]])  # Управляющий сигнал (отклонение руля высоты)

# Выполнение одного шага симуляции
state, reward, terminated, truncated, info = env.step(control_input)

print(f'🎯 Управляющее воздействие: {control_input.flatten()[0]}')
print(f'📊 Новое состояние: {state.flatten()}')
print(f'🏆 Награда: {reward:.6f}')
print(f'🔚 Завершено: {terminated}')
print(f'⏰ Прервано: {truncated}')

/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


## 📊 Анализ результатов

Исследуем внутреннее состояние модели и опорные сигналы:

In [5]:
# Просмотр истории управляющих воздействий
print(f'📝 История управляющих воздействий:')
env.unwrapped.model.store_input

array([[1., 0., 0., ..., 0., 0., 0.]])

In [6]:
# Проверка опорного сигнала для угла тангажа (theta)
print(f'🎯 Текущее значение опорного сигнала для theta:')
env.unwrapped.reference_signal[3][1]  # Четвертое состояние (theta) на втором временном шаге

0.0

In [7]:
# Анализ полученной награды
print(f'🏆 Детальный анализ награды:')
print(f'   • Значение: {reward:.2e}')
print(f'   • Тип: {type(reward)}')
print(f'   • Интерпретация: Чем меньше значение, тем лучше отслеживание опорного сигнала')

reward

8.243965275209204e-06

## 🎉 Заключение

В этом примере мы успешно:

✅ **Создали среду Boeing 747** с правильными параметрами
✅ **Настроили временные параметры** и опорный сигнал
✅ **Выполнили симуляцию** с управляющим воздействием
✅ **Проанализировали результаты** и внутреннее состояние модели

### 📈 Состояния модели:
- **u**: Продольная скорость
- **w**: Вертикальная скорость
- **q**: Угловая скорость тангажа
- **theta**: Угол тангажа

### 🎮 Управление:
- **Вход**: Отклонение руля высоты
- **Цель**: Отслеживание заданного угла тангажа

Модель готова для использования в задачах управления и обучения с подкреплением! 🚀